<h1>VIT Testing - tamncheese Jason Kahei Tam<h1>


Import

In [2]:
import torch
import torch.nn as nn
import os
from torch.utils.data import DataLoader, Dataset
from transformers import ViTImageProcessor, ViTForImageClassification, AutoFeatureExtractor, AutoModel, \
    AutoModelForImageClassification, AutoImageProcessor
import matplotlib.pyplot as plt
from PIL import Image
import cv2
import numpy as np
from torchvision import datasets, transforms
import pandas as pd


C:\Users\jason\Documents\OMSCS\DSGT\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


CPU or CUDA


In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cpu')

Prepare Dataset

In [4]:
file_list = []
for root, dirs, files, in os.walk("training"):
    for file in files:
        file_list.append(file)
file_list

train_df = pd.read_csv("training.csv")

In [5]:


class FungiDataset(Dataset):
    def __init__(self, df, extractor, img_dir, transform = None):
        self.df = df
        self.transform = transform
        self.img_dir = img_dir
        self.extractor = extractor

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.df.iloc[idx]["image_path"])
        label = self.df.iloc[idx]["poisonous"]
        # print(img_path)
        image = Image.open(img_path).convert('RGB')

        if self.transform:
            image = self.transform(image)

        image = self.extractor(images=image, return_tensors="pt")["pixel_values"].squeeze(0)
        return image, torch.tensor(label, dtype = torch.long)



Configure DINOv2


In [6]:
extractor = AutoImageProcessor.from_pretrained("facebook/dinov2-base")
model = AutoModelForImageClassification.from_pretrained('facebook/dinov2-base', num_labels=2)

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.48, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.
Some weights of Dinov2ForImageClassification were not initialized from the model checkpoint at facebook/dinov2-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Load Data


In [7]:
img_dir = "training"
train_dataset = FungiDataset(train_df, extractor, img_dir)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

Train the Model


In [8]:
optimizer = torch.optim.Adam(model.parameters(), lr = 0.00001)
criterion = nn.CrossEntropyLoss()

epochs = 10
for epoch in range(epochs):
    print(f"Epoch {epoch + 1}/{epochs}")

    model.train()
    train_loss = 0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images).logits
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
    avg_train_loss = train_loss / len(train_loader)
    print(f"Training Loss: {avg_train_loss} for epoch {epoch + 1}/{epochs}")



Epoch 1/10
Training Loss: 1.0184929370880127 for epoch 1/10
Epoch 2/10
Training Loss: 0.17603765428066254 for epoch 2/10
Epoch 3/10
Training Loss: 0.09964486956596375 for epoch 3/10
Epoch 4/10
Training Loss: 0.021811870858073235 for epoch 4/10
Epoch 5/10
Training Loss: 0.003398667089641094 for epoch 5/10
Epoch 6/10
Training Loss: 0.0008448290172964334 for epoch 6/10
Epoch 7/10
Training Loss: 0.000329566013533622 for epoch 7/10
Epoch 8/10
Training Loss: 0.00017590999777894467 for epoch 8/10
Epoch 9/10
Training Loss: 0.00011596946569625288 for epoch 9/10
Epoch 10/10
Training Loss: 8.796634938335046e-05 for epoch 10/10
